In [ ]:
#use niwrap2912 conda environment
# Import required libraries
import numpy as np
import nibabel as nib
from scipy.ndimage import map_coordinates
import os
import matplotlib.pyplot as plt
import re
from scipy.stats import zscore

# Configuration
#for exvivo (bigbrain stuff is on the next cell!)
base_path = '/Users/dennis.jungchildmind.org/Desktop/exvivo_postslurm_wb/output_wb_120um_method1'
workbench_base_path ='/Users/dennis.jungchildmind.org/Desktop/exvivo_postslurm_wb'
manual_base_path = '/Users/dennis.jungchildmind.org/Desktop/exvivo_postslurm'
output_dir = './figure'
hemi = 'lh'
fontsize = 8
percentil_low = 20
percentil_high = 80
cmap = 'plasma'
resolutions = ['120', '240']
data_types = ['raw', 'raw_zscore', 'diff', 'diff_zscore']

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get all subject folders
subject_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
print(f"Found {len(subject_folders)} subjects: {subject_folders}")

def load_and_process_data(file_path, sorted_order=None):
    """Load data and compute processed versions (diff, zscore)."""
    data = np.load(file_path, allow_pickle=True)
    
    # Use provided sorted order or compute new one
    if sorted_order is None:
        sorted_order = np.argsort(np.mean(data['all_values'], axis=0))
    
    # Extract and sort raw values
    raw = data['all_values'][:, sorted_order]
    
    # Compute processed versions
    raw_zscore = zscore(raw, axis=0)
    
    diff = np.diff(raw, axis=0)
    diff_zscore = zscore(diff, axis=0)
    
    return {
        'raw': raw,
        'raw_zscore': raw_zscore,
        'diff': diff,
        'diff_zscore': diff_zscore,
        'sorted_order': sorted_order,
        'data': data
    }

def set_colorbar_limits(data, data_type):
    """Set appropriate colorbar limits based on data type."""
    if data_type == 'raw':
        return np.percentile(data, percentil_low), np.percentile(data, percentil_high)
    #elif data_type == 'diff'
    elif data_type in ['diff','diff_zscore','raw_zscore']:
        p_low, p_high = np.percentile(data, percentil_low), np.percentile(data, percentil_high)
        max_clim = round(max(abs(p_low), abs(p_high)), 2)
        return p_low, p_high#-max_clim, max_clim
    #elif data_type in ['diff_zscore', 'raw_zscore']:
    #    max_clim = 1
    #    return -max_clim, max_clim

def setup_colorbar(im, ax, data_type, vmin, vmax):
    """Setup colorbar with appropriate ticks and labels."""
    cbar = plt.colorbar(im, ax=ax)
    
    if data_type in ['diff', 'diff_zscore']:
        #max_clim = min(abs(vmin), abs(vmax))
        cbar.set_ticks([vmin, 0, vmax])
        cbar.set_ticklabels([f'{vmin:.2f}', '0', f'{vmax:.2f}'])
    
    return cbar

def create_subplot(ax, data, title, data_type, plot_dist_values):
    """Create a single subplot with proper formatting."""
    im = ax.imshow(data, aspect='auto', 
                   extent=[0, data.shape[1], plot_dist_values[-1], plot_dist_values[0]], 
                   cmap=cmap)
    
    vmin, vmax = set_colorbar_limits(data, data_type)

    im.set_clim(vmin, vmax)
    
    setup_colorbar(im, ax, data_type, vmin, vmax)
    
    ax.set_xlabel('Vertices (ordered by column mean of diff)', fontsize=fontsize+1)
    ax.set_ylabel('Rel. Inf Surf (mm)', fontsize=fontsize)
    ax.set_title(title, fontsize=fontsize+1,fontweight='bold')
    
    return im

# Process each subject
for subject_name in subject_folders:
    print(f"\nProcessing subject: {subject_name}")
    
    # Initialize data storage dictionaries for this subject
    data_storage = {
        'manual': {res: {} for res in resolutions},
        'workbench': {res: {} for res in resolutions}
    }
    all_dist_values = {}
    
    try:
        # Load and process data for all resolutions
        for resolution in resolutions:
            print(f"  Processing resolution: {resolution}")
            
            # Define file paths
            workbench_path = f'{workbench_base_path}/output_wb_{resolution}_method1/{subject_name}/{hemi}/inf_{resolution}_method1_wb_raw_intensity.npz'
            manual_path = f'{manual_base_path}/output_{resolution}_max_960um_dist_method1/{subject_name}/{hemi}/inf_{resolution}_method1_manual_raw_intensity.npz'
            
            # Check if files exist
            if not os.path.exists(workbench_path):
                print(f"    Warning: Workbench file not found: {workbench_path}")
                continue
            if not os.path.exists(manual_path):
                print(f"    Warning: Manual file not found: {manual_path}")
                continue
            
            # Load manual data first to establish sorting order
            manual_results = load_and_process_data(manual_path)
            sorted_order = manual_results['sorted_order']
            
            # Load workbench data using the same sorting order
            workbench_results = load_and_process_data(workbench_path, sorted_order)
            
            # Store processed data
            data_storage['manual'][resolution] = manual_results
            data_storage['workbench'][resolution] = workbench_results
            
            # Extract distance array
            all_dist_values[resolution] = manual_results['data']['dist_array']
        
        # Skip if no data was loaded
        if not all_dist_values:
            print(f"  Skipping {subject_name}: No data files found")
            continue
        
        # Use the first resolution's distance values for plotting
        plot_dist_values = all_dist_values[resolutions[0]]
        
        # Create individual figures for each data type
        individual_figures = []
        for data_type in data_types:
            print(f"  Creating plots for data type: {data_type}")
            
            # Get data for both resolutions
            data1_manual = data_storage['manual']['120'][data_type]
            data2_manual = data_storage['manual']['240'][data_type]
            
            # Create figure with subplots
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 3))
            
            # Create subplots
            create_subplot(ax1, data1_manual, '120um', data_type, plot_dist_values)
            create_subplot(ax2, data2_manual, '240um', data_type, plot_dist_values)
            
            # Add tight layout
            plt.tight_layout()
            
            # Save individual figure
            individual_figures.append(fig)
            plt.close(fig)
        
        # Create combined 2x2 figure
        combined_fig, axes = plt.subplots(2, 2, figsize=(12, 6))
        axes = axes.flatten()
        
        # Copy each individual figure to the combined figure
        for i, (data_type, individual_fig) in enumerate(zip(data_types, individual_figures)):
            # Get the individual figure's canvas as an image
            individual_fig.canvas.draw()
            buf = np.frombuffer(individual_fig.canvas.buffer_rgba(), dtype=np.uint8)
            buf = buf.reshape(individual_fig.canvas.get_width_height()[::-1] + (4,))
            buf = buf[:, :, :3]  # Remove alpha channel
            
            # Display in the combined figure
            axes[i].imshow(buf)
            axes[i].set_title(f'{data_type}', fontsize=fontsize+2, fontweight='bold')
            axes[i].axis('off')
        
        # Adjust layout and save combined figure
        plt.tight_layout()
        output_path = os.path.join(output_dir, f'combined_comparison_120um_vs_240um_{subject_name}.png')
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close(combined_fig)
        print(f"  Saved figure: {output_path}")
        
        # Close individual figures to free memory
        for fig in individual_figures:
            plt.close(fig)
            
    except Exception as e:
        print(f"  Error processing {subject_name}: {str(e)}")
        continue

print(f"\nProcessing complete! Figures saved in {output_dir}")

Found 17 subjects: ['I57_new_confidence', 'I58_new_confidence', 'I45_new_confidence', 'I59_new_confidence', 'I56_new_confidence', 'I41_new_confidence', 'I54_new_confidence', 'I38_new_confidence', 'I53_new_confidence', 'I46_new_confidence', 'EXC022_new_confidence', 'I52_new_confidence', 'I48_new_confidence', 'I55_new_confidence', 'I61_new_confidence', 'KC001_new_confidence', 'I60_new_confidence']

Processing subject: I57_new_confidence
  Processing resolution: 120um
  Processing resolution: 240um
  Creating plots for data type: raw
  Creating plots for data type: raw_zscore
  Creating plots for data type: diff
  Creating plots for data type: diff_zscore
  Saved figure: ./figure/combined_comparison_120um_vs_240um_I57_new_confidence.png

Processing subject: I58_new_confidence
  Processing resolution: 120um
  Processing resolution: 240um
  Creating plots for data type: raw
  Creating plots for data type: raw_zscore
  Creating plots for data type: diff
  Creating plots for data type: diff_z

/opt/anaconda3/envs/niwrap3912/lib/python3.11/site-packages/matplotlib/axis.py:1540: RuntimeWarning: One of rtol or atol is not valid, atol: nan, rtol: 0
  mask = np.isclose(tr_minor_locs[:, None], tr_major_locs[None, :],


  Saved figure: ./figure/combined_comparison_120um_vs_240um_I55_new_confidence.png

Processing subject: I61_new_confidence
  Processing resolution: 120um
  Processing resolution: 240um
  Skipping I61_new_confidence: No data files found

Processing subject: KC001_new_confidence
  Processing resolution: 120um
  Processing resolution: 240um
  Skipping KC001_new_confidence: No data files found

Processing subject: I60_new_confidence
  Processing resolution: 120um
  Processing resolution: 240um
  Skipping I60_new_confidence: No data files found

Processing complete! Figures saved in ./figure


In [38]:
#for bigbrain
# Configuration
base_path = '/Users/dennis.jungchildmind.org/Desktop/exvivo_postslurm_wb/output_wb_120um_method1'
workbench_base_path ='/Users/dennis.jungchildmind.org/Desktop/exvivo_postslurm_wb'
manual_base_path = '/Users/dennis.jungchildmind.org/Desktop/exvivo_postslurm'
output_dir = './figure'
hemi = 'lh'
fontsize = 8
percentil_low = 20
percentil_high = 80
cmap = 'plasma'
resolutions = ['120', '240']
data_types = ['raw', 'raw_zscore', 'diff', 'diff_zscore']

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def load_and_process_data(file_path, sorted_order=None):
    """Load data and compute processed versions (diff, zscore)."""
    data = np.load(file_path, allow_pickle=True)
    
    # Use provided sorted order or compute new one
    if sorted_order is None:
        sorted_order = np.argsort(np.mean(data['all_values'], axis=0))
    
    # Extract and sort raw values
    raw = data['all_values'][:, sorted_order]
    
    # Compute processed versions
    raw_zscore = zscore(raw, axis=0)
    
    diff = np.diff(raw, axis=0)
    diff_zscore = zscore(diff, axis=0)
    
    return {
        'raw': raw,
        'raw_zscore': raw_zscore,
        'diff': diff,
        'diff_zscore': diff_zscore,
        'sorted_order': sorted_order,
        'data': data
    }

def set_colorbar_limits(data, data_type):
    """Set appropriate colorbar limits based on data type."""
    if data_type == 'raw':
        return np.percentile(data, percentil_low), np.percentile(data, percentil_high)
    #elif data_type == 'diff'
    elif data_type in ['diff','diff_zscore','raw_zscore']:
        p_low, p_high = np.percentile(data, percentil_low), np.percentile(data, percentil_high)
        max_clim = round(max(abs(p_low), abs(p_high)), 2)
        return p_low, p_high#-max_clim, max_clim
    #elif data_type in ['diff_zscore', 'raw_zscore']:
    #    max_clim = 1
    #    return -max_clim, max_clim

def setup_colorbar(im, ax, data_type, vmin, vmax):
    """Setup colorbar with appropriate ticks and labels."""
    cbar = plt.colorbar(im, ax=ax)
    
    if data_type in ['diff', 'diff_zscore']:
        #max_clim = min(abs(vmin), abs(vmax))
        cbar.set_ticks([vmin, 0, vmax])
        cbar.set_ticklabels([f'{vmin:.2f}', '0', f'{vmax:.2f}'])
    
    return cbar

def create_subplot(ax, data, title, data_type, plot_dist_values):
    """Create a single subplot with proper formatting."""
    im = ax.imshow(data, aspect='auto', 
                   extent=[0, data.shape[1], plot_dist_values[-1], plot_dist_values[0]], 
                   cmap=cmap)
    
    vmin, vmax = set_colorbar_limits(data, data_type)

    im.set_clim(vmin, vmax)
    
    setup_colorbar(im, ax, data_type, vmin, vmax)
    
    ax.set_xlabel('Vertices (ordered by column mean of diff)', fontsize=fontsize+1)
    ax.set_ylabel('Rel. Inf Surf (mm)', fontsize=fontsize)
    ax.set_title(title, fontsize=fontsize+1,fontweight='bold')
    
    return im

# Process each subject

# Initialize data storage dictionaries for this subject
data_storage = {
    'manual': {res: {} for res in resolutions},
    'workbench': {res: {} for res in resolutions}
}
all_dist_values = {}


# Load and process data for all resolutions
for resolution in resolutions:
    print(f"  Processing resolution: {resolution}")
    
    # Define file paths
    manual_path = f'/Users/dennis.jungchildmind.org/Desktop/BigBrain/PlosBiology2020gii/bigbrain_layer3_{resolution}um_method1_manual_raw_intensity.npz'
    workbench_path = f'/Users/dennis.jungchildmind.org/Desktop/BigBrain/PlosBiology2020gii/BigBrain_voxel_up_and_down/surf_voxel_up_and_down/{resolution}um_method1/bigbrain_workbench_raw_intensity.npz'


    #workbench_path = f'{workbench_base_path}/output_wb_{resolution}_method1/{subject_name}/{hemi}/inf_{resolution}um_method1_wb_raw_intensity.npz'
    #manual_path = f'{manual_base_path}/output_{resolution}_max_960um_dist_method1/{subject_name}/{hemi}/inf_{resolution}um_method1_manual_raw_intensity.npz'
    
    # Check if files exist
    if not os.path.exists(workbench_path):
        print(f"    Warning: Workbench file not found: {workbench_path}")
        continue
    if not os.path.exists(manual_path):
        print(f"    Warning: Manual file not found: {manual_path}")
        continue
    
    # Load manual data first to establish sorting order
    manual_results = load_and_process_data(manual_path)
    sorted_order = manual_results['sorted_order']
    
    # Load workbench data using the same sorting order
    workbench_results = load_and_process_data(workbench_path, sorted_order)
    
    # Store processed data
    data_storage['manual'][resolution] = manual_results
    data_storage['workbench'][resolution] = workbench_results
    
    # Extract distance array
    all_dist_values[resolution] = manual_results['data']['dist_array']
    print(all_dist_values[resolution])



# Use the first resolution's distance values for plotting
plot_dist_values = all_dist_values[resolutions[0]]

# Create individual figures for each data type
individual_figures = []
for data_type in data_types:
    print(f"  Creating plots for data type: {data_type}")
    
    # Get data for both resolutions
    data1_manual = data_storage['manual']['120'][data_type]
    data2_manual = data_storage['manual']['240'][data_type]
    
    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 3))
    
    # Create subplots
    create_subplot(ax1, data1_manual, '120um', data_type, plot_dist_values)
    create_subplot(ax2, data2_manual, '240um', data_type, plot_dist_values)
    
    # Add tight layout
    plt.tight_layout()
    
    # Save individual figure
    individual_figures.append(fig)
    plt.close(fig)

# Create combined 2x2 figure
combined_fig, axes = plt.subplots(2, 2, figsize=(12, 6))
axes = axes.flatten()

# Copy each individual figure to the combined figure
for i, (data_type, individual_fig) in enumerate(zip(data_types, individual_figures)):
    # Get the individual figure's canvas as an image
    individual_fig.canvas.draw()
    buf = np.frombuffer(individual_fig.canvas.buffer_rgba(), dtype=np.uint8)
    buf = buf.reshape(individual_fig.canvas.get_width_height()[::-1] + (4,))
    buf = buf[:, :, :3]  # Remove alpha channel
    
    # Display in the combined figure
    axes[i].imshow(buf)
    axes[i].set_title(f'{data_type}', fontsize=fontsize+2, fontweight='bold')
    axes[i].axis('off')

# Adjust layout and save combined figure
plt.tight_layout()
output_path = os.path.join(output_dir, f'combined_comparison_120um_vs_240um_bigbrain.png')
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.close(combined_fig)
print(f"  Saved figure: {output_path}")

# Close individual figures to free memory
for fig in individual_figures:
    plt.close(fig)
    
   

print(f"\nProcessing complete! Figures saved in {output_dir}")

  Processing resolution: 120
[ 1.08  0.96  0.84  0.72  0.6   0.48  0.36  0.24  0.12  0.   -0.12 -0.24
 -0.36 -0.48 -0.6  -0.72 -0.84 -0.96 -1.08]
  Processing resolution: 240


/var/folders/71/4g31r9017kq32wt_mx1khjdm0000gn/T/ipykernel_49247/2213912970.py:30: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  raw_zscore = zscore(raw, axis=0)
/var/folders/71/4g31r9017kq32wt_mx1khjdm0000gn/T/ipykernel_49247/2213912970.py:30: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  raw_zscore = zscore(raw, axis=0)
/var/folders/71/4g31r9017kq32wt_mx1khjdm0000gn/T/ipykernel_49247/2213912970.py:30: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  raw_zscore = zscore(raw, axis=0)
/var/folders/71/4g31r9017kq32wt_mx1khjdm0000gn/T/ipykernel_49247/2213912970.py:30: RuntimeWarning: Precision loss occurred in moment 

[ 0.96  0.72  0.48  0.24  0.   -0.24 -0.48 -0.72 -0.96]
  Creating plots for data type: raw
  Creating plots for data type: raw_zscore
  Creating plots for data type: diff
  Creating plots for data type: diff_zscore


/opt/anaconda3/envs/niwrap3912/lib/python3.11/site-packages/matplotlib/axis.py:1540: RuntimeWarning: One of rtol or atol is not valid, atol: nan, rtol: 0
  mask = np.isclose(tr_minor_locs[:, None], tr_major_locs[None, :],


  Saved figure: ./figure/combined_comparison_120um_vs_240um_bigbrain.png

Processing complete! Figures saved in ./figure


In [35]:
print(resolution)

240
